In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [10]:
indep.shape

(400, 3)

In [11]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param_grid = {
    'n_estimators': [100, 200, 300],             
    'max_features': ['sqrt', 'log2'],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [None,100],     
    'min_samples_split': [2],             
    'min_samples_leaf': [1],               
    'bootstrap': [True, False] }                
rf = RandomForestClassifier()
grid = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=3, scoring='accuracy')
grid.fit(indep, dep)
print("Best parameters found: ", grid.best_params_)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
best_rf = grid.best_estimator_
y_pred = best_rf.predict(indep)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best parameters found:  {'bootstrap': True, 'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best cross-validation score: 0.85


In [13]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(dep, y_pred)
print(cm)       

[[256   1]
 [  0 143]]


In [14]:
from sklearn.metrics import classification_report
clf_report = classification_report(dep, y_pred)

In [15]:
print(clf_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       257
           1       0.99      1.00      1.00       143

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400



In [16]:
print(cm)

[[256   1]
 [  0 143]]


In [17]:
grid.predict([[27,57000,0]])

C:\Users\swath\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0], dtype=int64)

In [18]:
re=grid.cv_results_

In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.327409,0.009135,0.012475,0.008421,True,gini,None,sqrt,1,2,100,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.7875,0.9375,0.8625,0.8375,0.8000,0.8450,0.053385,17
1,0.631596,0.011792,0.028796,0.003490,True,gini,None,sqrt,1,2,200,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.7875,0.9375,0.8750,0.8375,0.8000,0.8475,0.054429,8
2,0.927672,0.016002,0.039921,0.003962,True,gini,None,sqrt,1,2,300,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.8000,0.9375,0.8750,0.8375,0.7875,0.8475,0.054429,8
3,0.304588,0.005279,0.016122,0.005424,True,gini,None,log2,1,2,100,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.7875,0.9375,0.8625,0.8375,0.7875,0.8425,0.055678,29
4,0.635075,0.019034,0.027523,0.004940,True,gini,None,log2,1,2,200,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.8000,0.9375,0.8875,0.8375,0.7875,0.8500,0.055902,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.891145,0.026442,0.050466,0.004862,False,log_loss,100,sqrt,1,2,200,"{'bootstrap': False, 'criterion': 'log_loss', ...",0.7875,0.9125,0.8625,0.8125,0.7500,0.8250,0.057009,57
68,1.321087,0.021340,0.063910,0.007533,False,log_loss,100,sqrt,1,2,300,"{'bootstrap': False, 'criterion': 'log_loss', ...",0.7875,0.9125,0.8625,0.8125,0.7625,0.8275,0.053852,45
69,0.436078,0.027962,0.026867,0.007638,False,log_loss,100,log2,1,2,100,"{'bootstrap': False, 'criterion': 'log_loss', ...",0.7875,0.9250,0.8750,0.8125,0.7375,0.8275,0.065860,45
70,0.903294,0.013981,0.047500,0.007283,False,log_loss,100,log2,1,2,200,"{'bootstrap': False, 'criterion': 'log_loss', ...",0.7875,0.9125,0.8750,0.8125,0.7625,0.8300,0.055678,38
